In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/audio-features-csv/audio_features.csv
/kaggle/input/audiotest-csv/audioTest.csv
/kaggle/input/audio-test2-csv/audio_test2.csv


In [2]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Bidirectional, Dropout, BatchNormalization, Dense, MaxPooling1D, concatenate, GlobalAveragePooling1D,Flatten
from sklearn.metrics import confusion_matrix
from keras.models import save_model, load_model
from imblearn.under_sampling import RandomUnderSampler

2024-03-11 10:12:51.435860: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 10:12:51.435958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 10:12:51.669896: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv("/kaggle/input/audio-features-csv/audio_features.csv")
df.head()

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,origin_sample,LABEL
0,0.369246,0.114583,2329.765047,1668.270744,4210.475297,0.125022,-344.44583,74.368630,-26.999317,38.491272,...,-0.012907,-3.529766,-4.690998,-13.813165,-6.292424,-10.091806,-1.323195,-8.031024,LA_T_6747050,fake
1,0.261991,0.172123,1387.932080,1209.900252,2261.231024,0.083097,-257.60925,129.847320,-36.567460,33.844730,...,-12.913269,-7.028874,-6.108995,-24.280165,-14.908502,-14.017176,-6.970436,-15.313593,LA_T_6747050,fake
2,0.367695,0.070103,2301.121569,1604.363941,3903.382457,0.135931,-322.69806,86.057620,-30.443005,41.763557,...,5.834561,-1.498449,1.390241,-17.747652,-10.772357,-9.536011,-3.103892,-11.773501,LA_T_6747050,fake
3,0.630570,0.000076,3492.844688,2376.639078,6383.248901,0.171265,-786.05550,56.160923,-37.678963,50.892410,...,12.214974,-3.049933,13.535940,-7.106358,11.727142,-6.619684,5.530129,-3.223867,LA_T_6747050,fake
4,0.417924,0.091955,2470.647169,1780.860052,4330.865479,0.177080,-295.70105,67.723100,-19.865486,33.981766,...,3.750639,-6.569212,5.015835,-8.746722,-2.513248,-10.342720,1.080411,-9.256080,LA_T_7031111,fake


In [4]:
X = df.iloc[:, :-2]
y = df.iloc[:, -1]

In [5]:
lb = preprocessing.LabelBinarizer()
y = lb.fit_transform(y)
y = y.ravel()
print(y)

[0 0 0 ... 1 1 1]


In [6]:
X_train,X_Test,Y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)
X_train.shape[1]

26

In [7]:
rus = RandomUnderSampler(random_state=42)
X_train_res, Y_train_res = rus.fit_resample(X_train, Y_train)

In [8]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train_res)
X_test_std = scaler.transform(X_Test)

X_train_std = X_train_std.reshape((X_train_std.shape[0], X_train_std.shape[1], 1))
X_test_std = X_test_std.reshape((X_test_std.shape[0], X_test_std.shape[1], 1))

In [9]:
from tensorflow.keras.optimizers import Adam

def WIRENetSpoofImproved(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, 3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(256, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(512, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(512, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    
    x = GlobalAveragePooling1D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)  # Add dropout for regularization
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model


# Model instantiation
input_shape = (X_train_std.shape[1], 1)  # Adjust based on your data
model = WIRENetSpoofImproved(input_shape)
# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_std, Y_train_res, epochs=10, batch_size=32, validation_data=(X_test_std, y_test))

Epoch 1/10


I0000 00:00:1710151998.076290      99 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


503/503 [==============================] - 17s 13ms/step - loss: 0.5841 - accuracy: 0.7245 - val_loss: 0.5279 - val_accuracy: 0.7294
Epoch 2/10
503/503 [==============================] - 6s 11ms/step - loss: 0.4026 - accuracy: 0.8146 - val_loss: 0.3277 - val_accuracy: 0.8460
Epoch 3/10
503/503 [==============================] - 6s 12ms/step - loss: 0.3520 - accuracy: 0.8381 - val_loss: 0.3701 - val_accuracy: 0.8194
Epoch 4/10
503/503 [==============================] - 6s 11ms/step - loss: 0.3236 - accuracy: 0.8595 - val_loss: 0.2974 - val_accuracy: 0.8460
Epoch 5/10
503/503 [==============================] - 6s 11ms/step - loss: 0.3002 - accuracy: 0.8693 - val_loss: 0.3864 - val_accuracy: 0.8162
Epoch 6/10
503/503 [==============================] - 6s 12ms/step - loss: 0.2755 - accuracy: 0.8801 - val_loss: 0.3681 - val_accuracy: 0.8237
Epoch 7/10
503/503 [==============================] - 6s 12ms/step - loss: 0.2659 - accuracy: 0.8879 - val_loss: 0.2890 - val_accuracy: 0.8604
Epoch 8/1

In [10]:
model.save("wirenet_model.h5")

# Load the model
loaded_model = load_model("wirenet_model.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
evaluation = loaded_model.evaluate(X_test_std,y_test)
print("Evaluation Loss:", evaluation[0])
print("Evaluation Accuracy:", evaluation[1])

623/623 [==============================] - 2s 3ms/step - loss: 0.3287 - accuracy: 0.8686
Evaluation Loss: 0.32870179414749146
Evaluation Accuracy: 0.8686325550079346


In [13]:
# accuracy=history.history['accuracy'][-1]
# TP=history.history['true_positives'][-1]
# FP=history.history['false_positives'][-1]
# TN=history.history['true_negatives'][-1]
# FN=history.history['false_negatives'][-1]

# ERR = ( FP / (FP+TN) + FN / (FN+TP))/2
# print(ERR)

In [14]:
test_df = pd.read_csv('/kaggle/input/audiotest-csv/audioTest.csv')
X_test = test_df.drop(["origin_sample", "LABEL"], axis=1)
y_true = test_df["LABEL"]

# Encode the labels
y_true_encoded = lb.fit_transform(y_true)

# Standardize the features
scaler = StandardScaler()
X_test_std = scaler.fit_transform(X_test)
X_test_std = X_test_std.reshape((X_test_std.shape[0], X_test_std.shape[1], 1))

# Make predictions with your model
predictions = model.predict(X_test_std)
predictions_label = (predictions > 0.5).astype("int32")

# Calculate accuracy using the encoded labels
accuracy = accuracy_score(y_true_encoded, predictions_label)
print(f'Accuracy: {accuracy * 100:.2f}%')

5/5 [==============================] - 0s 60ms/step
Accuracy: 62.75%
